# Usage Demonstration
*Written by Chalermsak Chatdokmaiprai, CPE Dept., Kasetsart University, Thailand. First release April 27, 2020.*

To use the `reglang` module, import it.

In [6]:
from reglang import dfa, nfa, regx

## 1. Creating a DFA
Now we want to create a new `dfa` object that represents a DFA (deterministic finite automaton) that accepts binary strings that has an odd number of 1's. We first define its transition function `delta1`, which is a Python `dict` whose keys are tuples `(state, symbol)` and the mapped values are next states.

In [7]:
delta1 = {
    (1,'0') : 1,
    (1,'1') : 2,
    (2,'0') : 2,
    (2,'1') : 1
}

The above transition function shows that our DFA has only two states, namely 1 and 2. Next, we'll use the constructor function `dfa()` to create a `dfa` object `dfa1` with the above `delta1` dict as its transition function and 1 being the start state and {2} being the set of its final states. The transition function, the start state, and the set of final states are passed to the keyword parameters `delta`, `start`, and `finals`, respectively.

In [8]:
dfa1 = dfa(delta=delta1, start=1, finals={2})

We can print the `dfa` object just created to see all its components. Notice that the DFA's input alphabet `sigma` and the set of states `states` are automatically derived from the given transition function.

In [9]:
print(dfa1)

states = {1, 2}
sigma = {'0', '1'}
start = 1
finals = {2}
delta = {
    (1, '0') : 1
    (1, '1') : 2
    (2, '0') : 2
    (2, '1') : 1
}


We can use `dfa1` to test whether a string has an odd number of 1's using the `accept()` method.

In [10]:
dfa1.accept('011011')

False

In [11]:
dfa1.accept('1000')

True

When creating a new `dfa` object, the given delta function need not be defined for every symbol in the alphabet. A trap state `'_H_'` will automatically be created to make it a total function. The following DFA `dfa2` accepts the language (01 U 010)\*. The given transition function `delta2` specifies only essential transitions, omitting all transitions to the trap state. 

In [12]:
delta2 = {   # a DFA that accepts (01 U 010)*
    ('s', '0') : '0',
    ('0', '1') : '01',
    ('01', '0'): '010',
    ('010', '0'): '0',
    ('010', '1'): '01'
}
dfa2 = dfa(delta=delta2, start='s', finals={'s', '01', '010'}) 

Printing the DFA `dfa2` shows the completed transition function with an inserted trap state `'_H_'`.

In [13]:
print(dfa2)

states = {'0', '01', '010', '_H_', 's'}
sigma = {'0', '1'}
start = 's'
finals = {'01', '010', 's'}
delta = {
    ('0', '0') : '_H_'
    ('0', '1') : '01'
    ('01', '0') : '010'
    ('01', '1') : '_H_'
    ('010', '0') : '0'
    ('010', '1') : '01'
    ('_H_', '0') : '_H_'
    ('_H_', '1') : '_H_'
    ('s', '0') : '0'
    ('s', '1') : '_H_'
}


We test various input strings on `dfa2` below. Notice that the empty string `''` can be an input as well.

In [14]:
inputlist = ['0101001','010010010','01','010','','1010','010001','01101','010100100101']
for inpstr in inputlist:
    if dfa2.accept(inpstr):
        print("'{0}' accepted".format(inpstr))
    else:
        print("'{0}' rejected".format(inpstr))

'0101001' accepted
'010010010' accepted
'01' accepted
'010' accepted
'' accepted
'1010' rejected
'010001' rejected
'01101' rejected
'010100100101' accepted


## 2. Creating an NFA
An `nfa` object that represents an NFA (nondeterministic finite automaton) can be created in a similar way as the DFA, except that the transition function of an `nfa` maps each tuple `(state, symbol)` to a set of states and the `symbol` in each tuple may be an empty string `''`. The following code creates an `nfa` object that accepts the language $\{w \in {a,b,c}^*: w\ does\ not\ have\ some\ symbol\ in\ \{a, b, c\}\}$. The constructor function `nfa` is used to create an `nfa` object.

In [15]:
delta3 = {
    (0, '')  : {1, 2, 3},
    (1, 'a') : {1},
    (1, 'b') : {1},
    (2, 'a') : {2},
    (2, 'c') : {2},
    (3, 'b') : {3},
    (3, 'c') : {3}
}
nfa3 = nfa(delta=delta3, start=0, finals={1, 2, 3})

In [16]:
print(nfa3)

states = {0, 1, 2, 3}
sigma = {'a', 'b', 'c'}
start = 0
finals = {1, 2, 3}
delta = {
    (0, '') : {1, 2, 3}
    (1, 'a') : {1}
    (1, 'b') : {1}
    (2, 'a') : {2}
    (2, 'c') : {2}
    (3, 'b') : {3}
    (3, 'c') : {3}
}


In [17]:
nfa3.accept('bccba')

False

In [18]:
nfa3.accept('cbbc')

True

In [19]:
nfa3.accept('babbba')

True

In [20]:
nfa3.accept('ac')

True

## 3. Conversions Between DFAs and NFAs
### 3.1 From NFA to DFA
The `nfa` class has a method `to_dfa()` to produce an equivalent `dfa` object accepting the same language. Let's use it to produce a new `dfa` that is equivalent to the above `nfa3`.

In [21]:
dfa4 = nfa3.to_dfa()

In [22]:
print(dfa4)

states = {(), (0, 1, 2, 3), (1, 2), (1, 3), (1,), (2, 3), (2,), (3,)}
sigma = {'a', 'b', 'c'}
start = (0, 1, 2, 3)
finals = {(0, 1, 2, 3), (1, 2), (1, 3), (1,), (2, 3), (2,), (3,)}
delta = {
    ((), 'a') : ()
    ((), 'b') : ()
    ((), 'c') : ()
    ((0, 1, 2, 3), 'a') : (1, 2)
    ((0, 1, 2, 3), 'b') : (1, 3)
    ((0, 1, 2, 3), 'c') : (2, 3)
    ((1, 2), 'a') : (1, 2)
    ((1, 2), 'b') : (1,)
    ((1, 2), 'c') : (2,)
    ((1, 3), 'a') : (1,)
    ((1, 3), 'b') : (1, 3)
    ((1, 3), 'c') : (3,)
    ((1,), 'a') : (1,)
    ((1,), 'b') : (1,)
    ((1,), 'c') : ()
    ((2, 3), 'a') : (2,)
    ((2, 3), 'b') : (3,)
    ((2, 3), 'c') : (2, 3)
    ((2,), 'a') : (2,)
    ((2,), 'b') : ()
    ((2,), 'c') : (2,)
    ((3,), 'a') : ()
    ((3,), 'b') : (3,)
    ((3,), 'c') : (3,)
}


Notice that each state of the produced `dfa4` is a set of states in `nfa3` in the form of a tuple. This is a result of the conversion algorithm being used. The following expressions test the equivalence.

In [23]:
print(dfa4.accept('bccba'))
print(dfa4.accept('cbbc'))
print(dfa4.accept('babbba'))
print(dfa4.accept('ac'))

False
True
True
True


### 3.2 From DFA to NFA
Reversely, the `dfa` class has a method `to_nfa()` to produce an equivalent `nfa` though it's a very trivial task. As an example, let's go back to our DFA `dfa1`and convert it into an equivalent NFA `nfa5`.

In [24]:
print(dfa1)

states = {1, 2}
sigma = {'0', '1'}
start = 1
finals = {2}
delta = {
    (1, '0') : 1
    (1, '1') : 2
    (2, '0') : 2
    (2, '1') : 1
}


In [25]:
nfa5 = dfa1.to_nfa()

You can see how trivial the conversion is. The transtion function of `nfa5` is derived from that of `dfa1` by simply making each next state a set. 

In [26]:
print(nfa5)

states = {1, 2}
sigma = {'0', '1'}
start = 1
finals = {2}
delta = {
    (1, '0') : {1}
    (1, '1') : {2}
    (2, '0') : {2}
    (2, '1') : {1}
}


## 4. Operations on DFAs

### 4.1 Simplifying State Names
The DFA `dfa4` converted from an NFA has cumbersome state names. We can created another equivalent DFA with simple-numbered states by using the method `renumbered()`. It received an integer argument that is to become the starting number of state names. The following call sets the starting number of state names to 1.

In [27]:
dfa6 = dfa4.renumbered(1)

Note that the name of the start state of the produced DFA may or may not be the same as the given starting number. For example, the start state of the produced `dfa6` is not 1, but 2.

In [28]:
print(dfa6)

states = {1, 2, 3, 4, 5, 6, 7, 8}
sigma = {'a', 'b', 'c'}
start = 8
finals = {1, 2, 3, 4, 5, 7, 8}
delta = {
    (1, 'a') : 1
    (1, 'b') : 3
    (1, 'c') : 4
    (2, 'a') : 3
    (2, 'b') : 2
    (2, 'c') : 5
    (3, 'a') : 3
    (3, 'b') : 3
    (3, 'c') : 6
    (4, 'a') : 4
    (4, 'b') : 6
    (4, 'c') : 4
    (5, 'a') : 6
    (5, 'b') : 5
    (5, 'c') : 5
    (6, 'a') : 6
    (6, 'b') : 6
    (6, 'c') : 6
    (7, 'a') : 4
    (7, 'b') : 5
    (7, 'c') : 7
    (8, 'a') : 1
    (8, 'b') : 2
    (8, 'c') : 7
}


Another way to simplify state names of a DFA is to renumber them in a standard way using the method `standard_numbered()`. The state will be 1, 2, 3, ... and so on, where 1 is the DFA's start state. The sorted order of the input alphabet is used to determine the numbering of states.

In [29]:
print(dfa2)

states = {'0', '01', '010', '_H_', 's'}
sigma = {'0', '1'}
start = 's'
finals = {'01', '010', 's'}
delta = {
    ('0', '0') : '_H_'
    ('0', '1') : '01'
    ('01', '0') : '010'
    ('01', '1') : '_H_'
    ('010', '0') : '0'
    ('010', '1') : '01'
    ('_H_', '0') : '_H_'
    ('_H_', '1') : '_H_'
    ('s', '0') : '0'
    ('s', '1') : '_H_'
}


In [30]:
dfa7 = dfa2.standard_numbered()

In [31]:
print(dfa7)

states = {1, 2, 3, 4, 5}
sigma = {'0', '1'}
start = 1
finals = {1, 4, 5}
delta = {
    (1, '0') : 2
    (1, '1') : 3
    (2, '0') : 3
    (2, '1') : 4
    (3, '0') : 3
    (3, '1') : 3
    (4, '0') : 5
    (4, '1') : 3
    (5, '0') : 2
    (5, '1') : 4
}


Note that both `renumbered()` and `standard_numbered()` return a new `dfa` object. The original DFA stays intact. Also, as a side effect, the `dfa` returned from `standard_numbered()` will have all nonreachable states, if existing in the original DFA, eliminated. In that case the number of states of the returned DFA would be less than that of the original one.

### 4.2 Removal of Nonreachable States
A nonreachable state of a DFA cannot be reached from the start state, thus is useless in the operation of the DFA and can be removed. The following DFA `dfa8` has two nonreachable states. Can you guess which?

In [32]:
delta8 = {
    (1,'0') : 1,
    (1,'1') : 4,
    (2,'0') : 1,
    (2,'1') : 2,
    (3,'0') : 2,
    (3,'1') : 4,
    (4,'0') : 4,
    (4,'1') : 1
}
dfa8 = dfa(delta=delta8, start=1, finals={2,3,4})

In [33]:
print(dfa8)

states = {1, 2, 3, 4}
sigma = {'0', '1'}
start = 1
finals = {2, 3, 4}
delta = {
    (1, '0') : 1
    (1, '1') : 4
    (2, '0') : 1
    (2, '1') : 2
    (3, '0') : 2
    (3, '1') : 4
    (4, '0') : 4
    (4, '1') : 1
}


The method `remove_nonreachable()` returns a new equivalent DFA that is free of nonreachable states.

In [34]:
dfa9 = dfa8.remove_nonreachable()

In [35]:
print(dfa9)

states = {1, 4}
sigma = {'0', '1'}
start = 1
finals = {4}
delta = {
    (1, '0') : 1
    (1, '1') : 4
    (4, '0') : 4
    (4, '1') : 1
}


So we know now that the state 2 and 3 of `dfa8` are nonreachable, thus absent from the returned `dfa9`.

### 4.3 Minimizing DFAs
The method `minimized()` produces the minimized DFA equivalent to the given DFA. All nonreachable states, if exist, will also be removed.

In [36]:
delta10 = {
    (1, 'a') : 2,
    (1, 'b') : 4,
    (2, 'a') : 5,
    (2, 'b') : 3,
    (3, 'a') : 2,
    (3, 'b') : 6,
    (4, 'a') : 1,
    (4, 'b') : 5,
    (5, 'a') : 5,
    (5, 'b') : 5,
    (6, 'a') : 3,
    (6, 'b') : 5,
    (7, 'a') : 6,
    (7, 'b') : 8,
    (8, 'a') : 7,
    (8, 'b') : 3
}
dfa10 = dfa(delta=delta10, start=1, finals={1, 3, 7})

In [37]:
print(dfa10)

states = {1, 2, 3, 4, 5, 6, 7, 8}
sigma = {'a', 'b'}
start = 1
finals = {1, 3, 7}
delta = {
    (1, 'a') : 2
    (1, 'b') : 4
    (2, 'a') : 5
    (2, 'b') : 3
    (3, 'a') : 2
    (3, 'b') : 6
    (4, 'a') : 1
    (4, 'b') : 5
    (5, 'a') : 5
    (5, 'b') : 5
    (6, 'a') : 3
    (6, 'b') : 5
    (7, 'a') : 6
    (7, 'b') : 8
    (8, 'a') : 7
    (8, 'b') : 3
}


In [38]:
dfa11 = dfa10.minimized()

In [39]:
print(dfa11)

states = {(1, 3), (2,), (4, 6), (5,)}
sigma = {'a', 'b'}
start = (1, 3)
finals = {(1, 3)}
delta = {
    ((1, 3), 'a') : (2,)
    ((1, 3), 'b') : (4, 6)
    ((2,), 'a') : (5,)
    ((2,), 'b') : (1, 3)
    ((4, 6), 'a') : (1, 3)
    ((4, 6), 'b') : (5,)
    ((5,), 'a') : (5,)
    ((5,), 'b') : (5,)
}


Due to the minimization algorithm being used, the states of the returned minimized DFA are shown as a tuple of states in the original DFA. If you want simpler state names, you may use either `renumbered()` or `standard_numbered()` to beautify the resulting DFA.

In [40]:
dfa12 = dfa10.minimized().renumbered(0)

In [41]:
print(dfa12)

states = {0, 1, 2, 3}
sigma = {'a', 'b'}
start = 2
finals = {2}
delta = {
    (0, 'a') : 1
    (0, 'b') : 2
    (1, 'a') : 1
    (1, 'b') : 1
    (2, 'a') : 0
    (2, 'b') : 3
    (3, 'a') : 2
    (3, 'b') : 1
}


In [42]:
dfa13 = dfa10.minimized().standard_numbered()

In [43]:
print(dfa13)

states = {1, 2, 3, 4}
sigma = {'a', 'b'}
start = 1
finals = {1}
delta = {
    (1, 'a') : 2
    (1, 'b') : 3
    (2, 'a') : 4
    (2, 'b') : 1
    (3, 'a') : 1
    (3, 'b') : 4
    (4, 'a') : 4
    (4, 'b') : 4
}


You can see more detailed steps in the minimization algorithm by setting the `verbose` parameter to `True`.

In [44]:
dfa14 = dfa10.minimized(verbose=True)

First partitioning:
 (1, 3) (2, 4, 5, 6)
Next partitioning:
 (1, 3) (2,) (4, 6) (5,)
Final partitioning:
 (1, 3) (2,) (4, 6) (5,)


In [45]:
print(dfa14)

states = {(1, 3), (2,), (4, 6), (5,)}
sigma = {'a', 'b'}
start = (1, 3)
finals = {(1, 3)}
delta = {
    ((1, 3), 'a') : (2,)
    ((1, 3), 'b') : (4, 6)
    ((2,), 'a') : (5,)
    ((2,), 'b') : (1, 3)
    ((4, 6), 'a') : (1, 3)
    ((4, 6), 'b') : (5,)
    ((5,), 'a') : (5,)
    ((5,), 'b') : (5,)
}


### 4.4 Complementation of DFAs
Given a DFA that accepts a language L, the method `complement()` returns a new DFA that accept the complementary language $\{w\ :\ w\ \notin\ L\}$. For example, the DFA `dfa15` created below accepts all strings over the alphabet {a, b} that ends with `'ba'`.

In [46]:
delta15 = {
    (1, 'a') : 1,
    (1, 'b') : 2,
    (2, 'a') : 3,
    (2, 'b') : 2,
    (3, 'a') : 1,
    (3, 'b') : 2
}
dfa15 = dfa(delta=delta15, start=1, finals={3})

In [47]:
for s in ['b', 'ba', 'aabba', 'ab', 'baab']:
    print(s, dfa15.accept(s))

b False
ba True
aabba True
ab False
baab False


We can create a new DFA `dfa16` that accepts all strings over {a, b} that does not end with `'ba'`, ie. the complementary language of the language that `dfa15` accepts, by calling the `complement()` method on `dfa15`.

In [48]:
dfa16 = dfa15.complement()

In [49]:
print(dfa16)

states = {1, 2, 3}
sigma = {'a', 'b'}
start = 1
finals = {1, 2}
delta = {
    (1, 'a') : 1
    (1, 'b') : 2
    (2, 'a') : 3
    (2, 'b') : 2
    (3, 'a') : 1
    (3, 'b') : 2
}


Notice that the produced `dfa16` differs from `dfa15` only in that all the final states of `dfa15` are nonfinal states of `dfa16` and vice versa so that strings accepted by `dfa15` is rejected by `dfa16`, and vice versa.

In [50]:
for s in ['b', 'ba', 'aabba', 'ab', 'baab']:
    print(s, dfa16.accept(s))

b True
ba False
aabba False
ab True
baab True


## 5. Test For Equivalence of Two FAs

The method `equiv()` checks whether two FAs are equivalent, in the sense that they accept the same language. It returns True if they are equivalent, or False if they are not. The method applied to the first FA, which may be a DFA or an NFA, while the second FA (also either a DFA or an NFA) is passed as its argument.

### 5.1 Equivalence of Two DFAs
For example, the above `dfa1` accepts binary strings that have an odd number of 1's, while `dfa2` accepts binary strings in the language (01 U 010)\*. Therefore they are not equivalent.

In [51]:
dfa1.equiv(dfa2)

False

In [52]:
dfa2.equiv(dfa1)

False

Since the equivalency is a symmetric relation, it's not surprising that the above two expressions yield the same result.

As a contrary example, the above `dfa14` is obtained by minimizing `dfa10`, therefore the two DFAs must be equivalent (otherwise the minimization process would be wrong).

In [53]:
dfa10.equiv(dfa14)

True

Also the above `dfa11` and `dfa12` differs only by the names of states, so they are equivalent.

In [54]:
dfa11.equiv(dfa12)

True

### 5.2 Equivalence of Two NFAs
The following NFAs `nfa17` and `nfa18` both accepts the same language (01 U 010)\*. The method `equiv()` we have used to test DFA equivalence can also be used to test NFA equivalence.

In [55]:
delta17 = {
    (0, '0') : {1, 2},
    (1, '1') : {0},
    (2, '1') : {3},
    (3, '0') : {0}
}
nfa17 = nfa(delta=delta17, start=0, finals={0})

In [56]:
delta18 = {
    (0, '0') : {1},
    (1, '1') : {2},
    (2, '0') : {0},
    (2, '')  : {0}
}
nfa18 = nfa(delta=delta18, start=0, finals={0}) 

In [57]:
nfa17.equiv(nfa18)

True

Of course, `nfa3` we previously created is not equivalent to `nfa17` since they accept different languages.

In [58]:
nfa3.equiv(nfa17)

False

In [59]:
nfa17.equiv(nfa3)

False

### 5.3 Equivalence between a DFA and an NFA
A DFA and an NFA are equivalent if they accept the same language. The method `equiv()` can be used to test the equivalence between a DFA and an NFA in either order since the equivalency is symmetric.

The previously created `dfa2` accepts the language (01 U 010)\*, which is the same language that `nfa17` accepts. Therefore they are equivalent. The following two expressions are equally good for testing their equivalence.

In [60]:
dfa2.equiv(nfa17)

True

In [61]:
nfa17.equiv(dfa2)

True

To sum up, the same method `equiv()` can be used to test equivalence of any two FAs, either of the same type or of different types.

## 6. Creating Regular Expressions


Regular expressions are implemented as `regx` objects. The constructor method `regx()` is used to create a new `regx` object by passing a string that defines a regular expression. There are a number of meta-characters that have special meanings in such regular-expression defining strings, namely

      # denoting the empty string,
      @ denoting the empty set,
      U denoting the Union operator,
      * denoting the Kleene star, and
      () denoting the parentheses (for grouping).

All other characters are taken to be symbols in the alphabet of the language defined by the regular expression. The following instructions create the simplest `regx` objects for the language {a}, {b}, {*empty-string*}, and *the empty set*, and assign them to the variables a, b, e, and m, respectively.

In [62]:
a = regx('a')  # representing the language {a}

In [63]:
b = regx('b')  # representing the language {b}

In [64]:
e = regx('#')  # representing the language {empty-string}

In [65]:
m = regx('@')  # representing the empty set

Each created `regx` object is printable as its defining string.

In [66]:
print(a)

a


In [67]:
print(e)

#


In [68]:
print(m)

@


More complicated `regx` objects can be created by more complicated defining strings with embedded meta-characters. The following `regx` object defines the language $\{w \in \{a, b, c\}^*:\ w\ does\ not\ have\ ac\ as\ a\ substring\}$. Note the precedence of the operations, whereby the Kleene star $*$ precedes concatenation and concatenation precedes union.

In [69]:
r_no_ac = regx('c*(aUbc*)*')

In [70]:
print(r_no_ac)

c*((aUbc*)*)


It happens that the meta-character @ may almost never be used in practice but it is included only for completeness purpose. However, the meta-character #, denoting the empty string, is indispensable. For example, the following `regx` object defines the language

$$\{w \in \{0, 1\}^*: w\ does\ not\ contain\ three\ consecutive\ 1's\}$$.

In [71]:
r_no_111 = regx('(#U1U11)(0U01U011)*')

## 7. Testing If a Regular Expression Generates a String

A regular expression $r$ defines exactly one regular language $L(r)$. A regular expression $r$ is said to *generate* a string $w$ if $w \in L(r)$. The method `gen()` can be applied to a `regx` object to test if a string, passed as its argument, is generated by the regular expression. 

Recall that the regular expression `r_no_ac` above generates strings that does not contain the substring $ac$. The following examples test whether `r_no_ac` can generate the strings $ccabc$ and $cacb$.

In [72]:
r_no_ac.gen('ccabc')

True

In [73]:
r_no_ac.gen('cacb')

False

The operator `>` may be used instead of the method `gen()` to do the same thing.

In [74]:
r_no_ac > 'ccabc'

True

In [75]:
r_no_ac > 'cacb'

False

We can also check to see if the regular expression `r_no_ac` can generate the empty string `''`.

In [76]:
r_no_ac > ''

True

Note the different representions of the empty string in the defining string of a regular expression (`#` on the left-hand side) and the (Python) empty string to be tested (`''` on the right-hand side).

In [77]:
regx('#Uab*') > ''

True

## 8. Conversions Between FAs And Regular Expressions

### 8.1 From an FA to a Regular Expression
The method `to_regx()` can be used to convert an FA, either a DFA or an NFA, to a regular expression. 

For example, let's use the method `to_regx` to convert the previously created `nfa17` that accepts the language (01 U 010)* to a regular expression that defines the same language.

In [78]:
r_of_nfa17 = nfa17.to_regx()

In [79]:
print(r_of_nfa17)

#U0(10)*1U(((0U0(10)*10)1)(((00U00(10)*10)1)*))(0U00(10)*1)


In [80]:
nfa17.accept('01010')

True

In [81]:
r_of_nfa17 > '01010'

True

In [82]:
nfa17.accept('011')

False

In [83]:
r_of_nfa17 > '011'

False

The method `to_regx()` can also be used to convert a DFA to a regular expression. For example, the previously created `dfa15` is a DFA that accepts $\{w \in \{a, b\}:\ w\ ends\ with\ ba\}$. We can convert this DFA to a regular expression that defines the same language.

In [84]:
r_of_dfa15 = dfa15.to_regx()

In [85]:
print(r_of_dfa15)

a*bb*a((((bUaa*b)b*)a)*)


In [86]:
dfa15.accept('aaba')

True

In [87]:
r_of_dfa15 > 'aaba'

True

In [88]:
dfa15.accept('baa')

False

In [89]:
r_of_dfa15 > 'baa'

False

### 8.2 From a Regular Expression to an FA
The method `to_nfa()` can be used to convert a regular expression to an NFA. Let's use it to convert the regx object `r_no_ac` to an NFA that accepts the language that `r_no_ac` defines.

In [90]:
print(r_no_ac)

c*((aUbc*)*)


In [91]:
nfa_no_ac = r_no_ac.to_nfa()

In [92]:
print(nfa_no_ac)

states = {1, 10, 11, 12, 13, 15, 2, 3, 4, 5, 6, 7, 8, 9}
sigma = {'a', 'b', 'c'}
start = 3
finals = {15}
delta = {
    (1, 'c') : {2}
    (10, '') : {9, 11}
    (11, '') : {13}
    (12, '') : {5, 7}
    (13, '') : {12, 15}
    (2, '') : {1, 4}
    (3, '') : {1, 4}
    (4, '') : {12, 15}
    (5, 'a') : {6}
    (6, '') : {13}
    (7, 'b') : {8}
    (8, '') : {9, 11}
    (9, 'c') : {10}
}


In [93]:
r_no_ac > 'cabc'

True

In [94]:
nfa_no_ac.accept('cabc')

True

In [95]:
r_no_ac > 'bac'

False

In [96]:
nfa_no_ac.accept('bac')

False

We also have the method `to_dfa()` that converts a regular expression to a DFA. The resulting DFA is automatically minimized and nicely made standard-numbered. (Yes, it calls the `dfa` method `standard_numbered()` to do it). 

In [97]:
dfa_no_ac = r_no_ac.to_dfa()

In [98]:
print(dfa_no_ac)

states = {1, 2, 3}
sigma = {'a', 'b', 'c'}
start = 1
finals = {1, 2}
delta = {
    (1, 'a') : 2
    (1, 'b') : 1
    (1, 'c') : 1
    (2, 'a') : 2
    (2, 'b') : 1
    (2, 'c') : 3
    (3, 'a') : 3
    (3, 'b') : 3
    (3, 'c') : 3
}


In [99]:
dfa_no_ac.accept('cabc')

True

In [100]:
dfa_no_ac.accept('bac')

False

## 9. Test For Equivalence of Two Regular Expressions

The `regx` method `equiv()` can be used to test if two regular expressions are equivalent, meaning that they define the same regular language.

For example, the regular expressions (0U1)\* and 0\*(10\*)\* are equivalent as they define the same language {0, 1}*.

In [101]:
r91 = regx('(0U1)*')

In [102]:
r92 = regx('0*(10*)*')

In [103]:
r91.equiv(r92)

True

In [104]:
r91.equiv(regx('(0U1)*1'))

False

The operator `==` can also be used instead of `equiv()` to mean the same thing.

In [105]:
r91 == r92

True

In [106]:
r91 == regx('(0U1)*1')

False

## 10. Operations on Regular Expressions

### 10.1 Union
If `regx` objects `r1` and `r2` represent regular expressions that define the languages $L_1$ and $L_2$, respectively, then the expression `r1 | r2` produces a new `regx` object that defines the language $L_1 \cup L_2$. For example, the following two `regx` objects `r101` and `r102` define the same language, thus are equivalent.

In [107]:
r101 = regx('a') | regx('b')

In [108]:
r102 = regx('aUb')

In [109]:
r101 == r102

True

The following `regx` object defines the language $\{w \in \{a,b\}^*:\ w\ ends\ with\ aa\}$.

In [110]:
r103 = regx('(aUb)*aa')

The following `regx` object defines the language $\{w \in \{a,b\}^*:\ w\ ends\ with\ bb\}$.

In [111]:
r104 = regx('(aUb)*bb')

Thus the following `regx` object `r105` defines the union of the above two languages, which is the set of all strings that ends with either aa or bb.

In [112]:
r105 = r103 | r104

In [113]:
print(r105)

(((aUb)*)a)aU(((aUb)*)b)b


In [114]:
r105 > 'baa'

True

In [115]:
r105 > 'bbb'

True

In [116]:
r105 > 'aaba'

False

### 10.2 Concatenation
If `regx` objects `r1` and `r2` represent regular expressions that define the languages $L_1$ and $L_2$, respectively, then the expression `r1 & r2` produces a new `regx` object that defines the concatenation of $L_1$ and $L_2$, denoted $L_1 L_2$. For example, the following two `regx` objects `r106` and `r107` define the same language, thus are equivalent.

In [117]:
r106 = regx('a') & regx('b')

In [118]:
r107 = regx('ab')

In [119]:
r106 == r107

True

The following `regx` object defines the language $\{a,b\}^*$.

In [120]:
r108 = regx('(aUb)*')

The following `regx` object defines the language $\{ab\}$.

In [121]:
r109 = regx('ab')

Thus the following `regx` object `r1010` defines the concatenation of the above two language, which is the set of all strings that ends with $ab$.

In [122]:
r1010 = r108 & r109

In [123]:
print(r1010)

((aUb)*)ab


In [124]:
r1010 > 'bbab'

True

In [125]:
r1010 > 'bbb'

False

The following `r1011` defined more directly is certainly equivalent to `r1010`.

In [126]:
r1011 = regx('(aUb)*ab')

In [127]:
print(r1011)

(((aUb)*)a)b


In [128]:
r1010 == r1011

True

### 10.3 Kleene Star

If a `regx` object `r1` defines a language $L_1$, then the expression `r1.star()` returns a new `regx` object that defines the language $L_1^*$.

For example, the following `regx` object `r1012` defines the language $\{01, 010\}$.

In [129]:
r1012 = regx('01 U 010')

Then `r1012.star()` returns a new `regx` object that defines the language $\{01, 010\}^*$, thus can as well be defined by the regular expression $(01 \cup 010)^*$.

In [130]:
r1013 = r1012.star()

In [131]:
print(r1013)

(01U010)*


In [132]:
r1013 == regx('(01 U 010)*')

True

Since the priviously created `nfa` object `nfa17` accepts this same language, the regular expression obtained by conversion from this NFA must also be equivalent to `r1013`.

In [133]:
nfa17.to_regx() == r1013

True

### 10.4 Operator Precedence

The `regx` operators `|`, `&`, and `.star()` corresponds to the language operations union, concatenation, and Kleene star, respectively. By convention, Kleene star precedes concatenation, while concatenation precedes union. Parentheses override the precedence. 

All the three `regx` operators faithfully follow this convention. For example, if `r`, `s`, and `t` are `regx` objects that define the language $L_r$, $L_s$, and $L_t$, respectively, then the expression `r&s.star()|t` produces a `regx` object that defines the language $L_r L_s^* \cup L_t$, which means the same as $((L_r (L_s^*)) \cup L_t)$.

As an illustration, the following example shows two equivalent ways of constructing a `regx` object that represents the regular expression $c^*(a \cup bc^*)^*$.

In [134]:
r1014 = regx('c*(aUbc*)*')  # directly created from a defining string

In [135]:
a = regx('a')

In [136]:
b = regx('b')

In [137]:
c = regx('c')

In [138]:
r1015 = c.star() & (a|b&c.star()).star() # indirectly created from smaller regx objects

In [139]:
r1014 == r1015   # Are they equivalent?

True

Yes, they are indeed equivalent. 